In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import os
from sklearn import preprocessing

In [ ]:
## Fill missing values with mean and median

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
# GKnn Algorithm
def GKnn(impute_df):
    for i in range(len(empty_loc_cols)-1):

        
        ## Differentiate numerical and categorical data
        numerical_cols = impute_df._get_numeric_data().columns
        cols = impute_df.columns
        categorical_cols = list(set(cols) - set(numerical_cols))
        num_df = impute_df.loc[:,numerical_cols]        #numerical dataframe
        cat_df = impute_df.loc[:,categorical_cols]      #categorical dataframe
        
        
        num_ck=num_df.iloc[empty_loc_cols[i],:]
        num_cp=num_df.copy()
        num_cp.drop([i + 1], axis = 0)             #### depends on indexing

        cat_ck=cat_df.iloc[empty_loc_cols[i],:]
        cat_cp=cat_df.copy()
        cat_cp.drop([i + 1], axis = 0)             #### depends on indexing
        
        
        # Compare queue and reference queue subtraction
        t1=pd.DataFrame()
        t2=pd.DataFrame(data = None, columns=cat_cp.columns, index=cat_cp.index)
        
        for j in range(num_cp.index.size):
            temp=pd.Series(num_cp.iloc[j,:]-num_ck)
            t1=t1.append(temp,ignore_index=True)
            
            if(cat_cp.iloc[j,:].equals(cat_ck)):
                t2.iloc[j,:] = 1
            else:
                t2.iloc[j,:] = 0

        t1.reset_index(drop=True, inplace=True)
        t2.reset_index(drop=True, inplace=True)
        t = pd.concat([t1, t2], axis=1)

            
        mmax=t.abs().max().max()
        mmin=t.abs().min().min()
        rho=0.5

        #3, seeking correlation coefficient
        ksi=((mmin+rho*mmax)/(abs(t)+rho*mmax))

        #4, seeking relevance
        r=ksi.sum(axis=1)/ksi.columns.size

        #5, relevance ranking, get the result r3>r2>r1
        result=r.sort_values(ascending=False)

        #6 add result to NaN locations
        index = result.iloc[[0]].index[0]
        for k in range(len(empty_loc[0])-1):
            if(empty_loc_cols[i] == empty_loc[0][k]):
                impute_df.iloc[empty_loc[0][k], empty_loc[1][k]] = impute_df.iloc[index, empty_loc[1][k]]
    return impute_df

In [ ]:
#Compute NRMS

from sklearn import metrics
#Frobenius Fun
def frob(x):
    return np.linalg.norm(x, 'fro')

#NRMS
def nrms(original, imputed):
    x = frob(imputed - original) / frob(original)
    return x

In [ ]:
## import xlsx files as pandas dataframe
path = '../Course Project Datasets/Incomplete Datasets Without Labels/Spam'
complete_df = pd.read_excel('../Course Project Datasets/Original Datasets Without Labels/Spam.xlsx', header = None)
incomplete_df_name_list = []
incomplete_df_list = []
for filename in os.listdir(path):
    if filename.endswith('.xlsx'):
        incomplete_df_list.append(pd.read_excel(path+"/"+filename, header=None))
        incomplete_df_name_list.append(filename.split(".")[0])

In [ ]:

imputed_df_list = []
nrms_data = []
index = 0
for incomplete_df in incomplete_df_list:

    incomplete_df.reset_index()
    complete_df.reset_index()
    
    complete_df.columns=["F"+str(i) for i in range(complete_df.shape[1])]
    incomplete_df.columns=["F"+str(i) for i in range(incomplete_df.shape[1])]
    
    ## find empty locations
    empty_loc = np.where(pd.isnull(incomplete_df))
    empty_loc_cols = np.unique(empty_loc[0])
    
    ## Replace NaN values to mean values
    incomplete_df = DataFrameImputer().fit_transform(incomplete_df)
    
    ## GKnn
    imputed_df = GKnn(incomplete_df)
    
    ##Compute NRMS
    nrmser = nrms(complete_df, imputed_df)
    nrms_data.append({incomplete_df_name_list[index]: nrmser})
    
    imputed_df_list.append(imputed_df)
    index+=1
nrms_data

In [ ]:
#save imputed dataframes to xlsx
index = 0
for df in imputed_df_list:
    df.to_excel("./Imputed/Spam/"+incomplete_df_name_list[index]+".xlsx")
    index+=1

In [ ]:
# Make Dictionary of dataset_name : nrms_values
nrms_dict = {}
index = 0;
for imputed_df in imputed_df_list:
    nrmser = nrms(complete_df, imputed_df)
    nrms_dict.update( {incomplete_df_name_list[index]: nrmser} )
    index+=1
nrms_dict

In [ ]:
# Download Dictionary as csv file
import csv
with open('./NRMS_AE/Spam.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in nrms_dict.items():
       writer.writerow([key, value])

In [ ]:
nrms_data

In [ ]:
imputed_df_list